In [1]:
!cat .env

STAGE=ENGIE_PRODUCTION


In [2]:
Company.objects.filter(name__icontains='ITA')

<QuerySet [<Company: e9fce210-f652-4d7d-befe-32e02a8f0f60: GSO - UHE-ITA>, <Company: 69c4238a-fc42-4a16-a312-364ff4cd10c6: GSB - UHE-ITA>, <Company: d436bd3c-e145-44f2-a5ee-797a67110efd: GES - UHE-ITA>]>

In [3]:
from django.db.models import Q
import datetime
from openpyxl import Workbook
from tqdm.notebook import tqdm
from django.contrib.admin.utils import flatten
from datetime import datetime
from tqdm.notebook import tqdm
import json

In [4]:
def remove_empty_dict(properties):
    l=[i for i in properties if i]
    l=len(l)
#     l=str(len(l))
    
    return l

In [5]:
def get_properties(data):
    p=0
    if data != '':
        if 'property_intersections' in data:
            p=len(data['property_intersections'])
        elif 'propertyIntersections' in data:
            p=len(data['propertyIntersections'])
    else:
        p=0   
#     p=''
#     if data != '':
#         if 'property_intersections' in data:
#             p=str(len(data['property_intersections']))
#         elif 'propertyIntersections' in data:
#             p=str(len(data['propertyIntersections']))
#     else:
#         p='0'
        
    return p

In [6]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
#     print(values)
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)
    final_translation = {item['value']: item['name'] for item in item_translation}
#     print(final_translation)
    
    if type(value) is str:
        return final_translation[value]
    elif type(value) is list:
        return ', '.join(list([get_value(field, occ, a) for a in value]))

In [7]:
def get_value_api(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['apiName']==field])
#     print(item_translation)
    final_translation = {item['value']: item['name'] for item in item_translation}
#     print(final_translation)
    
    if type(value) is str:
        return final_translation[value]
    elif type(value) is list:
        return ', '.join(list([get_value_api(field, occ, a) for a in value]))

In [8]:
def get_value_inner(field, inner_field, occ, value):

    values = occ.form_fields['fields']
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == inner_field])
#     print(mid_translation)
    final_translation = {item['value']: item['name'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [9]:
def get_kind(kind):
    
    kind_dict={"705": "Educação Ambiental", "706": "Programa de Visitas"}
    
    return kind_dict[kind]

In [10]:
company=Company.objects.get(name='GES - UHE-ITA')
company

<Company: d436bd3c-e145-44f2-a5ee-797a67110efd: GES - UHE-ITA>

In [11]:
mes_do_relatorio = 12

In [11]:
occurrences=OccurrenceRecord.objects.filter(Q(operational_control__uuid='75b22f2d-414f-4f08-af03-7992ba99d4cb')|Q(operational_control__uuid='efc6c6e3-88e6-489b-bc29-ab8609eea7af'), datetime__year=2023).order_by('datetime')

In [12]:
occurrences.count()

1933

In [13]:
occ_agendamento1=OccurrenceType.objects.get(uuid='6004e68b-3a62-41e6-9c8e-188c47efdb0e')
occ_atendimento1=OccurrenceType.objects.get(uuid='e1b126f4-1b1c-482d-9192-0aff757bcf53')
occ_cadastro=OccurrenceType.objects.get(uuid='71702b2c-0d0c-41c1-b5c4-243a40785967')
occ_agendamento2=OccurrenceType.objects.get(uuid='040500ff-cea2-4d11-9cb6-830792b1d62f')
occ_atendimento2=OccurrenceType.objects.get(uuid='2607fa44-a397-4928-9080-b9adcb788e9f')

In [14]:
wb = Workbook()

excel=[]

ws_agendamento = wb.create_sheet(title='Agendamento')
ws_atendimento = wb.create_sheet(title='Atendimento')
# ws_cadastro = wb.create_sheet(title='Cadastro da parte interessada')

ws_agendamento.append(['Serial', 'Criado por', 'Data', 'Parte interessada', 'Responsável - Nome', 'Responsável - Email',
                       'Responsável - Telefone', 'Responsável - Função', 'Tipo de agendamento', 'Local',
                       'Número de adultos', 'Número de crianças', 'Modalidade', 'Outros', 'Tema', 'Descrição',
                       'Faixa Etária', 'Número total previsto', 'Modalidade - Participação', 'Outros - Participação',
                       'Descrição - Participação', 'Tipo de público previsto', 'Número total previsto - Participação',
                       'Data do agendamento', 'Hora', 'Duração', 'Período', 'Atendimento - Equipe', 'Atendimento - Pessoa',
                       'Observações', 'Tipo', 'Município'])

ws_atendimento.append(['Serial', 'Criado por', 'Data', 'Número do agendamento', 'Parte(s) interessada(s)',
                       'Série/Ano', 'Data de início', 'Horário de início', 'Duração', 'Período', 'Modalidade',
                       'Tema', 'Número de presentes', 'Ferramenta(s)', 'Descrição', 'Quantidade(s)',
                       'Assuntos Tratados','Nomes dos palestrantes Internos', 'Nomes dos palestrantes Externos',
                       'Observações', 'Tipo', 'Tipo de atendimento', 'Município'])

# ws_cadastro.append(['Serial', 'Criado por', 'Criado em', 'Categoria', 'Segmento/Ocupação (Pessoa física)', 'Segmento (Associação)',
#                    'Segmento (Empresa)', 'Segmento/Ocupação (Grupo organizado)', 'Segmento/Ocupação (Instituição de ensino)',
#                    'Segmento (Outro)', 'Nome', 'Sigla', 'RG', 'CPF', 'CNPJ (Associação)', 'CNPJ (Empresa)', 'CNPJ (Grupo organizado)',
#                    'CNPJ (Instituição de ensino)', 'Comunidade', 'Caixa Postal', 'Telefone', 'Email', 'Site', 'Rede social', 'WhatsApp',
#                    'Representantes Institucionais - Nome', 'Representantes Institucionais - RG', 'Representantes Institucionais - CPF',
#                    'Representantes Institucionais - Função', 'Representantes Institucionais - Email',
#                    'Representantes Institucionais - Celular', 'Observações'])

for a in tqdm(occurrences):
    try:
        if a.occurrence_type==occ_agendamento1:

#             print(a.uuid, a.form_data['records'])

            ws_agendamento.append([a.number, a.created_by.get_full_name(),
                                   a.datetime.strftime("%d/%m/%Y"),
                                   ', '.join(list(filter(None, list(OccurrenceRecord.objects.filter(uuid__in=a.form_data['records']).values_list('form_data__intitution_name', flat=True))))) if 'records' in a.form_data else None,
                                   a.form_data['responsible_name'] if 'responsible_name' in a.form_data else None,
                                   a.form_data['responsible_email'] if 'responsible_email' in a.form_data else None,
                                   a.form_data['responsible_phone'] if 'responsible_phone' in a.form_data else None,
                                   a.form_data['responsible_function'] if 'responsible_function' in a.form_data else None,
                                   get_value('Tipo de agendamento', occ_agendamento1, a.form_data['agendament_type']) if 'agendament_type' in a.form_data else None,
                                   get_value('Local', occ_agendamento1, a.form_data['local']) if 'local' in a.form_data else None,
                                   a.form_data['adults_number_descriptipn'] if 'adults_number_descriptipn' in a.form_data else None,
                                   a.form_data['kinds_number_descriptipn'] if 'kinds_number_descriptipn' in a.form_data else None,
                                   get_value('Modalidade', occ_agendamento1, a.form_data['service_modality']) if 'service_modality' in a.form_data else None,
                                   a.form_data['other_service_modality'] if 'other_service_modality' in a.form_data else None,
                                   get_value('Tema', occ_agendamento1, a.form_data['service_theme']) if 'service_theme' in a.form_data else None,
                                   a.form_data['other_description'] if 'other_description' in a.form_data else None,
                                   get_value('Faixa Etária', occ_agendamento1, a.form_data['expectative_audience_service']) if 'expectative_audience_service' in a.form_data else None,
                                   a.form_data['expectative_total_service'] if 'expectative_total_service' in a.form_data else None,
                                   get_value_api('eventsModality', occ_agendamento1, a.form_data['events_modality']) if 'events_modality' in a.form_data else None,
                                   a.form_data['other_modality'] if 'other_modality' in a.form_data else None,
                                   a.form_data['events_descripton'] if 'events_descripton' in a.form_data else None,
                                   a.form_data['expectative_audience'] if 'expectative_audience' in a.form_data else None,
                                   a.form_data['expectative_total'] if 'expectative_total' in a.form_data else None,
                                   '/'.join(reversed(a.form_data['date_and_hour'].split('T')[0].split('-'))) if 'date_and_hour' in a.form_data and not not a.form_data['date_and_hour'] else None,
                                   a.form_data['date_and_hour'].split('T')[1].split('.')[0].split(':')[0] + a.form_data['date_and_hour'].split('T')[1].split('.')[0].split(':')[1] if 'date_and_hour' in a.form_data and not not a.form_data['date_and_hour'] else None, # AQUI
                                   a.form_data['duration'].split(':')[0] if 'duration' in a.form_data else None,
                                   get_value('Unidade de referência', occ_agendamento1, a.form_data['period']) if 'period' in a.form_data else None,
                                   Firm.objects.get(uuid=a.form_data['team']).name if 'team' in a.form_data else None,
                                   User.objects.get(uuid=a.form_data['people']).get_full_name() if 'people' in a.form_data and a.form_data['people'] is not None else None,
                                   a.form_data['observation'] if 'observation' in a.form_data else None,
                                   get_kind(a.operational_control.kind), a.city.name if a.city else None
                                  ])

        if a.occurrence_type==occ_agendamento2:

    #         print(a.uuid, a.form_data['records'])

            ws_agendamento.append([a.number, a.created_by.get_full_name(),
                                   a.datetime.strftime("%d/%m/%Y"),                          
                                   ', '.join(list(filter(None, list(OccurrenceRecord.objects.filter(uuid__in=a.form_data['records']).values_list('form_data__intitution_name', flat=True))))) if 'records' in a.form_data else None,
                                   a.form_data['responsible_name'] if 'responsible_name' in a.form_data else None,
                                   a.form_data['responsible_email'] if 'responsible_email' in a.form_data else None,
                                   a.form_data['responsible_phone'] if 'responsible_phone' in a.form_data else None,
                                   a.form_data['responsible_function'] if 'responsible_function' in a.form_data else None,
                                   get_value('Tipo de agendamento', occ_agendamento2, a.form_data['agendament_type']) if 'agendament_type' in a.form_data else None,
                                   get_value('Local', occ_agendamento2, a.form_data['local']) if 'local' in a.form_data else None,
                                   a.form_data['adults_number_descriptipn'] if 'adults_number_descriptipn' in a.form_data else None,
                                   a.form_data['kinds_number_descriptipn'] if 'kinds_number_descriptipn' in a.form_data else None,
                                   get_value('Modalidade', occ_agendamento2, a.form_data['service_modality']) if 'service_modality' in a.form_data else None,
                                   a.form_data['other_service_modality'] if 'other_service_modality' in a.form_data else None,
                                   get_value('Tema', occ_agendamento2, a.form_data['service_theme']) if 'service_theme' in a.form_data else None,
                                   a.form_data['other_description'] if 'other_description' in a.form_data else None,
                                   get_value('Faixa Etária', occ_agendamento2, a.form_data['expectative_audience_service']) if 'expectative_audience_service' in a.form_data else None,
                                   a.form_data['expectative_total_service'] if 'expectative_total_service' in a.form_data else None,
                                   get_value_api('eventsModality', occ_agendamento2, a.form_data['events_modality']) if 'events_modality' in a.form_data else None,
                                   a.form_data['other_modality'] if 'other_modality' in a.form_data else None,
                                   a.form_data['events_descripton'] if 'events_descripton' in a.form_data else None,
                                   a.form_data['expectative_audience'] if 'expectative_audience' in a.form_data else None,
                                   a.form_data['expectative_total'] if 'expectative_total' in a.form_data else None,
                                   '/'.join(reversed(a.form_data['date_and_hour'].split('T')[0].split('-'))) if 'date_and_hour' in a.form_data and not not a.form_data['date_and_hour'] else None,
                                   a.form_data['date_and_hour'].split('T')[1].split('.')[0].split(':')[0] + a.form_data['date_and_hour'].split('T')[1].split('.')[0].split(':')[1] if 'date_and_hour' in a.form_data and not not a.form_data['date_and_hour'] else None, # AQUI
                                   a.form_data['duration'].split(':')[0] if 'duration' in a.form_data else None,
                                   get_value('Unidade de referência', occ_agendamento2, a.form_data['period']) if 'period' in a.form_data else None,
                                   Firm.objects.get(uuid=a.form_data['team']).name if 'team' in a.form_data else None,
                                   User.objects.get(uuid=a.form_data['people']).get_full_name() if 'people' in a.form_data and a.form_data['people'] is not None else None,
                                   a.form_data['observation'] if 'observation' in a.form_data else None,
                                    get_kind(a.operational_control.kind), a.city.name if a.city else None
                                  ])

        if a.occurrence_type==occ_atendimento1:

    #         print(a.form_data['speakers'])

            ws_atendimento.append([a.number, a.created_by.get_full_name(),
                                  a.datetime.strftime("%d/%m/%Y"),
                                  ', '.join(list(a.number for a in OccurrenceRecord.objects.filter(uuid__in=a.form_data['records']))) if 'records' in a.form_data else None,
                                  ', '.join(list(OccurrenceRecord.objects.filter(uuid__in = a.form_data['records_two']).values_list('form_data__intitution_name', flat = True))) if 'records_two' in a.form_data else None,
                                  get_value('Série/Ano', occ_atendimento1, a.form_data['grade']) if 'grade' in a.form_data else None,
                                  '/'.join(reversed(a.form_data['start_hour'].split('T')[0].split('-'))) if 'start_hour' in a.form_data and not not a.form_data['start_hour'] else None,
                                  a.form_data['start_hour'].split('T')[1].split('.')[0].split(':')[0] + a.form_data['start_hour'].split('T')[1].split('.')[0].split(':')[1] if 'start_hour' in a.form_data and not not a.form_data['start_hour'] else None, # AQUI
                                  a.form_data['duration'] if 'duration' in a.form_data else None,
                                  get_value('Unidade de referência', occ_atendimento1, a.form_data['period']) if 'period' in a.form_data else None,
                                  get_value('Modalidade', occ_atendimento1, a.form_data['modality']) if 'modality' in a.form_data else None, 
                                  get_value('Tema', occ_atendimento1, a.form_data['theme']) if 'theme' in a.form_data else None,
                                  a.form_data['present_number'] if 'present_number' in a.form_data else None,
                                  ', '.join([get_value_inner('Ferramentas utilizadas', 'Ferramenta', occ_atendimento1, item['tool']) for item in a.form_data['tools_used'] if item and 'tool' in item]) if 'tools_used' in a.form_data else None,
                                  ', '.join([item['description'] for item in a.form_data['tools_used'] if item and 'description' in item]) if 'tools_used' in a.form_data else None,
                                  ', '.join([str(item['quantity']) for item in a.form_data['tools_used'] if item and 'quantity' in item]) if 'tools_used' in a.form_data else None,
                                  ', '.join([item['subjects_add'] for item in a.form_data['subjects'] if item and 'subjects_add' in item]) if 'subjects' in a.form_data else None,
                                  ', '.join([User.objects.get(uuid=item['speaker_name']).get_full_name() for item in a.form_data['speakers'] if item and 'speaker_name' in item and item['speaker_name'] is not None]) if 'speakers' in a.form_data else None,
                                  ', '.join([item['external_speak_name'] for item in a.form_data['speakers'] if item and 'external_speak_name' in item]) if 'speakers' in a.form_data else None,
                                  a.form_data['notes'] if 'notes' in a.form_data else None,
                                  get_kind(a.operational_control.kind),
                                  get_value('Tipo', occ_atendimento1, a.form_data['type']) if 'type' in a.form_data else None,
                                  a.city.name if a.city else None
                                  ])

        if a.occurrence_type==occ_atendimento2:

    #         print(a.form_data['speakers'])

            ws_atendimento.append([a.number,  a.created_by.get_full_name(),
                                  a.datetime.strftime("%d/%m/%Y"),                         
                                  ', '.join(list(a.number for a in OccurrenceRecord.objects.filter(uuid__in=a.form_data['records']))) if 'records' in a.form_data else None,
                                  ', '.join(list(OccurrenceRecord.objects.filter(uuid__in = a.form_data['records_two']).values_list('form_data__intitution_name', flat = True)))if 'records_two' in a.form_data else None,
                                  get_value('Série/Ano', occ_atendimento2, a.form_data['grade']) if 'grade' in a.form_data else None,
                                  '/'.join(reversed(a.form_data['start_hour'].split('T')[0].split('-'))) if 'start_hour' in a.form_data and not not a.form_data['start_hour'] else None,
                                  a.form_data['start_hour'].split('T')[1].split('.')[0].split(':')[0] + a.form_data['start_hour'].split('T')[1].split('.')[0].split(':')[1] if 'start_hour' in a.form_data and not not a.form_data['start_hour'] else None, # AQUI
                                  a.form_data['duration'] if 'duration' in a.form_data else None,
                                  get_value('Unidade de referência', occ_atendimento2, a.form_data['period']) if 'period' in a.form_data else None,
                                  get_value('Modalidade', occ_atendimento2, a.form_data['modality']) if 'modality' in a.form_data else None, 
                                  get_value('Tema', occ_atendimento2, a.form_data['theme']) if 'theme' in a.form_data else None,
                                  a.form_data['present_number'] if 'present_number' in a.form_data else None,
                                  ', '.join([get_value_inner('Ferramentas utilizadas', 'Ferramenta', occ_atendimento2, item['tool']) for item in a.form_data['tools_used'] if item and 'tool' in item]) if 'tools_used' in a.form_data else None,
                                  ', '.join([item['description'] for item in a.form_data['tools_used'] if item and 'description' in item]) if 'tools_used' in a.form_data else None,
                                  ', '.join([str(item['quantity']) for item in a.form_data['tools_used'] if item and 'quantity' in item]) if 'tools_used' in a.form_data else None,
                                  ', '.join([item['subjects_add'] for item in a.form_data['subjects'] if item and 'subjects_add' in item]) if 'subjects' in a.form_data else None,
                                  ', '.join([User.objects.get(uuid=item['speaker_name']).get_full_name() for item in a.form_data['speakers'] if item and 'speaker_name' in item and item['speaker_name'] is not None]) if 'speakers' in a.form_data else None,
                                  ', '.join([item['external_speak_name'] for item in a.form_data['speakers'] if item and 'external_speak_name' in item]) if 'speakers' in a.form_data else None,
                                  a.form_data['notes'] if 'notes' in a.form_data else None,
                                  get_kind(a.operational_control.kind),
                                  get_value('Tipo', occ_atendimento1, a.form_data['type']) if 'type' in a.form_data else None,
                                  a.city.name if a.city else None
                                  ])

    #     if a.occurrence_type==occ_cadastro:

    #         ws_cadastro.append([a.number, a.created_by.get_full_name(), a.created_at.strftime("%d/%m/%Y"),
    #                            get_value('Categoria', occ_cadastro, a.form_data['category']),
    #                            get_value_api('individual', occ_cadastro, a.form_data['individual']) if 'individual' in a.form_data else None,
    #                            a.form_data['association'] if 'association' in a.form_data else None,
    #                            a.form_data['legal_entity'] if 'legal_entity' in a.form_data else None,
    #                            get_value_api('organizedGroup', occ_cadastro, a.form_data['organized_group']) if 'organized_group' in a.form_data else None,
    #                            get_value_api('teachingIntitution', occ_cadastro, a.form_data['teaching_intitution']) if 'teaching_intitution' in a.form_data else None,
    #                            a.form_data['other_institution'] if 'other_institution' in a.form_data else None,
    #                            a.form_data['intitution_name'] if 'intitution_name' in a.form_data else None,
    #                            a.form_data['abbreviation_name'] if 'abbreviation_name' in a.form_data else None,
    #                            a.form_data['general_register'] if 'general_register' in a.form_data else None,
    #                            a.form_data['personal_register'] if 'personal_register' in a.form_data else None,
    #                            a.form_data['association_register'] if 'association_register' in a.form_data else None,
    #                            a.form_data['legal_entity_register'] if 'legal_entity_register' in a.form_data else None,
    #                            a.form_data['organized_group_register'] if 'organized_group_register' in a.form_data else None,
    #                            a.form_data['teaching_intitution_register'] if 'teaching_intitution_register' in a.form_data else None,
    #                            a.form_data['description_community'] if 'description_community' in a.form_data else None,
    #                            a.form_data['description_mailbox'] if 'description_mailbox' in a.form_data else None,
    #                            a.form_data['description_phone'] if 'description_phone' in a.form_data else None,
    #                            a.form_data['description_email'] if 'description_email' in a.form_data else None,
    #                            a.form_data['description_web_site'] if 'description_web_site' in a.form_data else None,
    #                            a.form_data['description_social_network'] if 'description_social_network' in a.form_data else None,
    #                            a.form_data['description_whatsapp_contact'] if 'description_whatsapp_contact' in a.form_data else None,
    #                            ', '.join([item['representative_name'] for item in a.form_data['institutional_representaties'] if item and 'representative_name' in item]) if 'institutional_representaties' in a.form_data else None,
    #                            ', '.join([item['rg'] for item in a.form_data['institutional_representaties'] if item and 'rg' in item]) if 'institutional_representaties' in a.form_data else None,
    #                            ', '.join([item['cpf'] for item in a.form_data['institutional_representaties'] if item and 'cpf' in item]) if 'institutional_representaties' in a.form_data else None,
    #                            ', '.join([item['representative_function'] for item in a.form_data['institutional_representaties'] if item and 'representative_function' in item]) if 'institutional_representaties' in a.form_data else None,
    #                            ', '.join([item['email'] for item in a.form_data['institutional_representaties'] if item and 'email' in item]) if 'institutional_representaties' in a.form_data else None,
    #                            ', '.join([item['phone_number'] for item in a.form_data['institutional_representaties'] if item and 'phone_number' in item]) if 'institutional_representaties' in a.form_data else None,
    #                            a.form_data['observation'] if 'observation' in a.form_data else None])
    except Exception as e:
        print(f'{a.number}\t{e}')

  0%|          | 0/1933 [00:00<?, ?it/s]

UHIT-RG-2023.1677	Firm matching query does not exist.


In [15]:
filename='Occorrencias_CDA_Anual_2023'
wb.save(filename=filename + '.xlsx')